# Preprocessing sessions

In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt  
% matplotlib inline
import random
from datetime import datetime
import seaborn as sns

In [2]:
df = pd.read_csv("sessions.csv")
df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [3]:
df.describe()

,secs_elapsed
count,1.043171e+07
mean,1.940581e+04
std,8.888424e+04
min,0.000000e+00
25%,2.290000e+02
50%,1.147000e+03
75%,8.444000e+03
max,1.799977e+06


## Delete users with invalid age from file *trai

In [15]:
df2.head(30)

action_type                action_detail  \
user_id    action                                                           
d1mm9tcy42 lookup                        NaN                          NaN   
           search_results              click          view_search_results   
           lookup                        NaN                          NaN   
           search_results              click          view_search_results   
           lookup                        NaN                          NaN   
           search_results              click          view_search_results   
           lookup                        NaN                          NaN   
           personalize                  data      wishlist_content_update   
           index                        view          view_search_results   
           lookup                        NaN                          NaN   
           search_results              click          view_search_results   
           lookup                        NaN                          NaN   
           personalize                  data      wishlist_content_update   
           index                        view          view_search_results   
           similar_listings             data             similar_listings   
           ajax_refresh_subtotal       click  change_trip_characteristics   
           similar_listings             data             similar_listings   
           ajax_refresh_subtotal       click  change_trip_characteristics   
           show                          NaN                          NaN   
           personalize                  data      wishlist_content_update   
           show                         view                           p3   
           lookup                        NaN                          NaN   
           search_results              click          view_search_results   
           lookup                        NaN                          NaN   
           search_results              click          view_search_results   
           personalize                  data      wishlist_content_update   
           lookup                        NaN                          NaN   
           index                        view          view_search_results   
           show                          NaN                          NaN   
           personalize                  data      wishlist_content_update   

                                      device_type  secs_elapsed  
user_id    action                                                
d1mm9tcy42 lookup                 Windows Desktop         319.0  
           search_results         Windows Desktop       67753.0  
           lookup                 Windows Desktop         301.0  
           search_results         Windows Desktop       22141.0  
           lookup                 Windows Desktop         435.0  
           search_results         Windows Desktop        7703.0  
           lookup                 Windows Desktop         115.0  
           personalize            Windows Desktop         831.0  
           index                  Windows Desktop       20842.0  
           lookup                 Windows Desktop         683.0  
           search_results         Windows Desktop       59274.0  
           lookup                 Windows Desktop          95.0  
           personalize            Windows Desktop        1399.0  
           index                  Windows Desktop       74886.0  
           similar_listings       Windows Desktop         255.0  
           ajax_refresh_subtotal  Windows Desktop        8134.0  
           similar_listings       Windows Desktop         183.0  
           ajax_refresh_subtotal  Windows Desktop      175570.0  
           show                   Windows Desktop          86.0  
           personalize            Windows Desktop        1535.0  
           show                   Windows Desktop       83251.0  
           lookup                 Windows Desktop 